### Based on https://developer.hashicorp.com/vault/tutorials/kubernetes/kubernetes-minikube-tls

In [1]:
%%bash
eval $(crc oc-env)
oc login -u kubeadmin https://api.crc.testing:6443

Logged into "https://api.crc.testing:6443" as "kubeadmin" using existing credentials.

You have access to 66 projects, the list has been suppressed. You can list all projects with 'oc projects'

Using project "default".


In [2]:
%%bash
helm repo add hashicorp https://helm.releases.hashicorp.com
helm repo update

"hashicorp" already exists with the same configuration, skipping
Hang tight while we grab the latest from your chart repositories...
...Successfully got an update from the "secrets-store-csi-driver" chart repository
...Successfully got an update from the "hashicorp" chart repository
...Successfully got an update from the "datadog" chart repository
...Successfully got an update from the "jetstack" chart repository
...Successfully got an update from the "bitnami" chart repository
Update Complete. ⎈Happy Helming!⎈


In [3]:
%%bash
rm -rf /tmp/vault
mkdir /tmp/vault

### OpenShift CA in CRC does not work. For that reason, I'm creating a CSR and signing it with an external CA

In [ ]:
%%bash
export VAULT_K8S_NAMESPACE="vault" \
export VAULT_HELM_RELEASE_NAME="vault" \
export VAULT_SERVICE_NAME="vault-internal" \
export K8S_CLUSTER_NAME="cluster.local" \
export WORKDIR=/tmp/vault

# Create KEY
openssl genrsa -out ${WORKDIR}/vault.key 2048
# Create CSR
cat > ${WORKDIR}/vault-csr.conf <<EOF
[req]
default_bits = 2048
prompt = no
encrypt_key = yes
default_md = sha256
distinguished_name = kubelet_serving
req_extensions = v3_req
[ kubelet_serving ]
O = system:nodes
CN = system:node:*.${VAULT_HELM_RELEASE_NAME}.svc.${K8S_CLUSTER_NAME}
[ v3_req ]
basicConstraints = CA:FALSE
keyUsage = nonRepudiation, digitalSignature, keyEncipherment, dataEncipherment
extendedKeyUsage = serverAuth, clientAuth
subjectAltName = @alt_names
[alt_names]
DNS.1 = *.${VAULT_SERVICE_NAME}
DNS.2 = *.${VAULT_SERVICE_NAME}.${VAULT_HELM_RELEASE_NAME}.svc.${K8S_CLUSTER_NAME}
DNS.3 = *.${VAULT_HELM_RELEASE_NAME}
DNS.4 = *.${VAULT_HELM_RELEASE_NAME}.svc.${K8S_CLUSTER_NAME}
IP.1 = 127.0.0.1
EOF

openssl req -new -key ${WORKDIR}/vault.key -out ${WORKDIR}/vault.csr -config ${WORKDIR}/vault-csr.conf


openssl ca -config openssl_local.cfg -cert cacert.pem -keyfile private/cakey.pem -in /tmp/vault/vault.csr -out certs/server.pem -md sha256

In [10]:
%env WORKDIR=/tmp/vault
%env VAULT_K8S_NAMESPACE=vault
%env VAULT_HELM_RELEASE_NAME=vault
%env VAULT_SERVICE_NAME=vault-internal 
%env K8S_CLUSTER_NAME=cluster.local 

env: WORKDIR=/tmp/vault
env: VAULT_K8S_NAMESPACE=vault
env: VAULT_HELM_RELEASE_NAME=vault
env: VAULT_SERVICE_NAME=vault-internal
env: K8S_CLUSTER_NAME=cluster.local


In [109]:
%%bash
kubectl create secret generic vault-tls \
   -n $VAULT_K8S_NAMESPACE \
   --from-file=vault.key=${WORKDIR}/vault.key \
   --from-file=vault.crt=/Users/jose/Demo/OC-test/demoCA/certs/server.pem \
   --from-file=vault.ca=/Users/jose/Demo/OC-test/demoCA/cacert.pem

secret/vault-tls created


### Adding enterprise license based on https://developer.hashicorp.com/vault/docs/platform/k8s/helm/enterprise

In [12]:
%%bash 
# Path to 
secret=$(cat /Users/jose/Demo/Vault_ADP/ADP_PlayGround/vault.hclic)
kubectl create secret generic vault-ent-license --from-literal="license=${secret}" -n $VAULT_K8S_NAMESPACE


secret/vault-ent-license created


In [133]:
%%bash
### TLS signed by an external CA
cat > ${WORKDIR}/overrides.yaml <<EOF
global:
   enabled: true
   tlsDisable: false # Disabling TLS to avoid issues when connecting to Vault via port forwarding
   openshift: true
injector:
   enabled: true
   #repository: registry.connect.redhat.com/hashicorp/vault-k8s
   # Testing on Mac M1 so moving to arm image
   image:
      repository: docker.io/hashicorp/vault-k8s
      #repository: registry.connect.redhat.com/hashicorp/vault-k8s
      # Testing on Mac M1 so moving to arm image
   agentImage:
      repository: docker.io/hashicorp/vault
      #repository: registry.connect.redhat.com/hashicorp/vault
      # Testing on Mac M1 so moving to arm image

# Supported log levels include: trace, debug, info, warn, error
logLevel: "trace" # Set to trace for initial troubleshooting, info for normal operation
server:
# config.yaml
   image:
      #repository: registry.connect.redhat.com/hashicorp/vault-enterprise
      #tag: 1.12.5-ent-ubi
      # Testing on Mac M1 so moving to arm image
      repository: docker.io/hashicorp/vault-enterprise
      tag: 1.13.1-ent
   enterpriseLicense:
      secretName: vault-ent-license
   extraEnvironmentVars:
      VAULT_CACERT: /vault/userconfig/vault-tls/vault.ca
      VAULT_TLSCERT: /vault/userconfig/vault-tls/vault.crt
      VAULT_TLSKEY: /vault/userconfig/vault-tls/vault.key
   volumes:
      - name: userconfig-vault-tls
        secret:
         defaultMode: 420
         secretName: vault-tls
   volumeMounts:
      - mountPath: /vault/userconfig/vault-tls
        name: userconfig-vault-tls
        readOnly: true
   standalone:
      enabled: false
   affinity: ""    # Affinity rules removed since I have a single node
   logLevel: "trace"
   ha:
      enabled: true
      replicas: 3 # we are going to install 3 nodes
      raft:
         enabled: true
         setNodeId: true
         config: |
            ui = true
            api_addr = "https://POD_IP:8200"
            listener "tcp" {
               tls_disable = 0
               address = "[::]:8200"
               cluster_address = "[::]:8201"
               tls_cert_file = "/vault/userconfig/vault-tls/vault.crt"
               tls_key_file  = "/vault/userconfig/vault-tls/vault.key"
               tls_client_ca_file = "/vault/userconfig/vault-tls/vault.ca"
            }
            
            storage "raft" {
               path = "/vault/data"
            
               retry_join {
                  auto_join             = "provider=k8s namespace=vault label_selector=\"component=server,app.kubernetes.io/name=vault\""
                  auto_join_scheme      = "https"
                  leader_ca_cert_file   = "/vault/userconfig/vault-tls/vault.ca"
                  leader_tls_servername = "vault-0.vault-internal" #Tiene que matchear una SAN del certificado
               }
            
            }

            # See https://developer.hashicorp.com/vault/tutorials/auto-unseal/autounseal-azure-keyvault
            # seal "azurekeyvault" {
            #    client_id      = "YOUR-APP-ID"
            #    client_secret  = "YOUR-APP-PASSWORD"
            #    tenant_id      = "YOUR-AZURE-TENANT-ID"
            #    vault_name     = "Test-vault-XXXXXX"
            #    key_name       = "generated-key"
            # }

            disable_mlock = true
            service_registration "kubernetes" {}
EOF


In [134]:
%%bash
helm install -n $VAULT_K8S_NAMESPACE $VAULT_HELM_RELEASE_NAME hashicorp/vault -f ${WORKDIR}/overrides.yaml

W0330 19:40:37.036254   25633 warnings.go:70] would violate PodSecurity "restricted:v1.24": allowPrivilegeEscalation != false (container "sidecar-injector" must set securityContext.allowPrivilegeEscalation=false), unrestricted capabilities (container "sidecar-injector" must set securityContext.capabilities.drop=["ALL"]), runAsNonRoot != true (pod or container "sidecar-injector" must set securityContext.runAsNonRoot=true), seccompProfile (pod or container "sidecar-injector" must set securityContext.seccompProfile.type to "RuntimeDefault" or "Localhost")
W0330 19:40:37.123373   25633 warnings.go:70] would violate PodSecurity "restricted:v1.24": allowPrivilegeEscalation != false (container "vault" must set securityContext.allowPrivilegeEscalation=false), unrestricted capabilities (container "vault" must set securityContext.capabilities.drop=["ALL"]), runAsNonRoot != true (pod or container "vault" must set securityContext.runAsNonRoot=true), seccompProfile (pod or container "vault" must se

NAME: vault
LAST DEPLOYED: Thu Mar 30 19:40:36 2023
NAMESPACE: vault
STATUS: deployed
REVISION: 1
NOTES:
Thank you for installing HashiCorp Vault!

Now that you have deployed Vault, you should look over the docs on using
Vault with Kubernetes available here:

https://www.vaultproject.io/docs/


Your release is named vault. To learn more about the release, try:

  $ helm status vault
  $ helm get manifest vault


In [137]:
%%bash
oc -n $VAULT_K8S_NAMESPACE get pods

NAME                                    READY   STATUS    RESTARTS   AGE
vault-0                                 0/1     Running   0          6s
vault-1                                 0/1     Running   0          6s
vault-2                                 0/1     Running   0          6s
vault-agent-injector-6cdd674474-7j9t6   0/1     Running   0          6s


## Initialize Vault

In [138]:
%%bash
kubectl exec -n $VAULT_K8S_NAMESPACE vault-0 -- vault operator init \
    -key-shares=1 \
    -key-threshold=1 \
    -format=json > ${WORKDIR}/cluster-keys.json


In [139]:
%%bash
jq -r ".unseal_keys_b64[]" ${WORKDIR}/cluster-keys.json
VAULT_UNSEAL_KEY=$(jq -r ".unseal_keys_b64[]" ${WORKDIR}/cluster-keys.json)

Y17M52JV7/wYsJqJI1mhaZ924Vx4HR/9re8QFBQSwkQ=


In [140]:
%%bash
kubectl exec -n $VAULT_K8S_NAMESPACE vault-0 -- vault operator unseal $(jq -r ".unseal_keys_b64[]" ${WORKDIR}/cluster-keys.json)

Key                     Value
---                     -----
Seal Type               shamir
Initialized             true
Sealed                  false
Total Shares            1
Threshold               1
Version                 1.13.1+ent
Build Date              2023-03-23T20:09:57Z
Storage Type            raft
Cluster Name            vault-cluster-f7ed5cc3
Cluster ID              dc10cd4b-86e1-e93d-8b10-6fc287f0560e
HA Enabled              true
HA Cluster              n/a
HA Mode                 standby
Active Node Address     <none>
Raft Committed Index    65
Raft Applied Index      63


In [141]:
%%bash
kubectl exec -n $VAULT_K8S_NAMESPACE -ti vault-1 -- vault operator unseal $(jq -r ".unseal_keys_b64[]" ${WORKDIR}/cluster-keys.json)

Unable to use a TTY - input is not a terminal or the right kind of file


Key                Value
---                -----
Seal Type          shamir
Initialized        true
Sealed             true
Total Shares       1
Threshold          1
Unseal Progress    0/1
Unseal Nonce       n/a
Version            1.13.1+ent
Build Date         2023-03-23T20:09:57Z
Storage Type       raft
HA Enabled         true


In [142]:
%%bash
kubectl exec -n $VAULT_K8S_NAMESPACE -ti vault-2 -- vault operator unseal $(jq -r ".unseal_keys_b64[]" ${WORKDIR}/cluster-keys.json)

Unable to use a TTY - input is not a terminal or the right kind of file


Key                Value
---                -----
Seal Type          shamir
Initialized        true
Sealed             true
Total Shares       1
Threshold          1
Unseal Progress    0/1
Unseal Nonce       n/a
Version            1.13.1+ent
Build Date         2023-03-23T20:09:57Z
Storage Type       raft
HA Enabled         true


In [143]:
%%bash
kubectl exec -n $VAULT_K8S_NAMESPACE -ti vault-0 -- vault status

Unable to use a TTY - input is not a terminal or the right kind of file


Key                     Value
---                     -----
Seal Type               shamir
Initialized             true
Sealed                  false
Total Shares            1
Threshold               1
Version                 1.13.1+ent
Build Date              2023-03-23T20:09:57Z
Storage Type            raft
Cluster Name            vault-cluster-f7ed5cc3
Cluster ID              dc10cd4b-86e1-e93d-8b10-6fc287f0560e
HA Enabled              true
HA Cluster              https://vault-0.vault-internal:8201
HA Mode                 active
Active Since            2023-03-30T17:41:03.766854116Z
Raft Committed Index    96
Raft Applied Index      96
Last WAL                31


In [144]:
%%bash
kubectl exec -n $VAULT_K8S_NAMESPACE -ti vault-0 --  vault license inspect


Unable to use a TTY - input is not a terminal or the right kind of file


Source: /vault/license/license (path from the VAULT_LICENSE_PATH environment variable)
Product: vault
License ID: 152295e1-3784-2641-bd01-bfdbc0a37c35
Customer ID: d9a6d7e8-e947-ea92-ec07-82c6901283c1
Installation ID: *
Issue Time: 2022-12-19 08:50:24.624461496 +0000 UTC
Start Time: 2022-12-19 00:00:00 +0000 UTC
Expiration Time: 2023-12-19 23:59:59.999 +0000 UTC
Termination Time: 2023-12-19 23:59:59.999 +0000 UTC
{"license_id":"152295e1-3784-2641-bd01-bfdbc0a37c35","customer_id":"d9a6d7e8-e947-ea92-ec07-82c6901283c1","installation_id":"*","issue_time":"2022-12-19T08:50:24.624461496Z","start_time":"2022-12-19T00:00:00Z","expiration_time":"2023-12-19T23:59:59.999Z","termination_time":"2023-12-19T23:59:59.999Z","flags":{"modules":["multi-dc-scale","governance-policy","advanced-data-protection-transform","advanced-data-protection-key-management"]},"features":["HSM","Performance Replication","DR Replication","MFA","Sentinel","Seal Wrapping","Control Groups","Performance Standby","Namespaces

In [145]:
%%bash
kubectl exec -n $VAULT_K8S_NAMESPACE -ti vault-1 -- vault status

Unable to use a TTY - input is not a terminal or the right kind of file


Key                                    Value
---                                    -----
Seal Type                              shamir
Initialized                            true
Sealed                                 false
Total Shares                           1
Threshold                              1
Version                                1.13.1+ent
Build Date                             2023-03-23T20:09:57Z
Storage Type                           raft
Cluster Name                           vault-cluster-f7ed5cc3
Cluster ID                             dc10cd4b-86e1-e93d-8b10-6fc287f0560e
HA Enabled                             true
HA Cluster                             https://vault-0.vault-internal:8201
HA Mode                                standby
Active Node Address                    https://10.217.1.92:8200
Performance Standby Node               true
Performance Standby Last Remote WAL    0
Raft Committed Index                   103
Raft Applied Index                     103


In [146]:
%%bash
kubectl exec -n $VAULT_K8S_NAMESPACE -ti vault-1 --  vault license inspect

Unable to use a TTY - input is not a terminal or the right kind of file


Source: /vault/license/license (path from the VAULT_LICENSE_PATH environment variable)
Product: vault
License ID: 152295e1-3784-2641-bd01-bfdbc0a37c35
Customer ID: d9a6d7e8-e947-ea92-ec07-82c6901283c1
Installation ID: *
Issue Time: 2022-12-19 08:50:24.624461496 +0000 UTC
Start Time: 2022-12-19 00:00:00 +0000 UTC
Expiration Time: 2023-12-19 23:59:59.999 +0000 UTC
Termination Time: 2023-12-19 23:59:59.999 +0000 UTC
{"license_id":"152295e1-3784-2641-bd01-bfdbc0a37c35","customer_id":"d9a6d7e8-e947-ea92-ec07-82c6901283c1","installation_id":"*","issue_time":"2022-12-19T08:50:24.624461496Z","start_time":"2022-12-19T00:00:00Z","expiration_time":"2023-12-19T23:59:59.999Z","termination_time":"2023-12-19T23:59:59.999Z","flags":{"modules":["multi-dc-scale","governance-policy","advanced-data-protection-transform","advanced-data-protection-key-management"]},"features":["HSM","Performance Replication","DR Replication","MFA","Sentinel","Seal Wrapping","Control Groups","Performance Standby","Namespaces

In [147]:
%%bash
kubectl exec -n $VAULT_K8S_NAMESPACE -ti vault-2 -- vault status

Unable to use a TTY - input is not a terminal or the right kind of file


Key                                    Value
---                                    -----
Seal Type                              shamir
Initialized                            true
Sealed                                 false
Total Shares                           1
Threshold                              1
Version                                1.13.1+ent
Build Date                             2023-03-23T20:09:57Z
Storage Type                           raft
Cluster Name                           vault-cluster-f7ed5cc3
Cluster ID                             dc10cd4b-86e1-e93d-8b10-6fc287f0560e
HA Enabled                             true
HA Cluster                             https://vault-0.vault-internal:8201
HA Mode                                standby
Active Node Address                    https://10.217.1.92:8200
Performance Standby Node               true
Performance Standby Last Remote WAL    0
Raft Committed Index                   108
Raft Applied Index                     108


In [148]:
%%bash
kubectl exec -n $VAULT_K8S_NAMESPACE -ti vault-2 --  vault license inspect

Unable to use a TTY - input is not a terminal or the right kind of file


Source: /vault/license/license (path from the VAULT_LICENSE_PATH environment variable)
Product: vault
License ID: 152295e1-3784-2641-bd01-bfdbc0a37c35
Customer ID: d9a6d7e8-e947-ea92-ec07-82c6901283c1
Installation ID: *
Issue Time: 2022-12-19 08:50:24.624461496 +0000 UTC
Start Time: 2022-12-19 00:00:00 +0000 UTC
Expiration Time: 2023-12-19 23:59:59.999 +0000 UTC
Termination Time: 2023-12-19 23:59:59.999 +0000 UTC
{"license_id":"152295e1-3784-2641-bd01-bfdbc0a37c35","customer_id":"d9a6d7e8-e947-ea92-ec07-82c6901283c1","installation_id":"*","issue_time":"2022-12-19T08:50:24.624461496Z","start_time":"2022-12-19T00:00:00Z","expiration_time":"2023-12-19T23:59:59.999Z","termination_time":"2023-12-19T23:59:59.999Z","flags":{"modules":["multi-dc-scale","governance-policy","advanced-data-protection-transform","advanced-data-protection-key-management"]},"features":["HSM","Performance Replication","DR Replication","MFA","Sentinel","Seal Wrapping","Control Groups","Performance Standby","Namespaces

## Get Root Token

In [149]:
%%bash
cat ${WORKDIR}/cluster-keys.json | jq -r ".root_token"

hvs.wqeNTydheKyHWe3N9OrPeOjD
